# Required Libraries and Modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ref_curve_task1 import ref
import dynamics as dyn
from Reg_Newton_Method import Newton_reg
from plots import traj_gen_plot
from plots import traj_gen_plot,traj_track_plot,track_error_plot
from mpc import mpc_controller1,mpc_controller2




# Allow Ctrl-C to work despite plotting
import signal

signal.signal(signal.SIGINT, signal.SIG_DFL)

#######################################
# Parameters from dynamics
#######################################

# Number of states
ns = dyn.ns
# Number of inputs
ni = dyn.ni

# Reference curve for trajectory generation

In [2]:
#######################################
# Trajectory parameters
#######################################

# Final time in seconds
tf = 20
# discretization step from dynamics
dt = dyn.dt   
# discrete-time samples
T = int(tf/dt) 

######################################
# Reference curve
######################################

# Flag to use a step reference or a cubic polynomial transition between two equilibria
step_ref = False

# Two equilibria for x1
x1_eq1_deg = -20
x1_eq2_deg = 30

# Reference curve between the two equilibria
xref,uref = ref(step_ref,T,x1_eq1_deg,x1_eq2_deg)

# Trajectory generation with Newton's method

In [3]:
#######################################
# Newton algorithm parameters
#######################################

# Maximum number of iterations of Newton's method
maxiters = 100
fixed_stepsize = 1 # not used

# Armijo parameters
stepsize_0 = 1
c = 0.5
beta = 0.7
armijo_maxiters = 20 
term_cond = 1e-5 

#############################################################
# Weight matrices for cost function for trajectory generation
#############################################################

Qt = np.diag([1000,1000,10,10])
Rt = 1*np.eye(ni)
QT = Qt

######################################
# Initial guess for the Newton method
######################################

xinit = np.repeat(xref[:,0].reshape(-1,1), T, axis=1)
uinit = np.repeat(uref[:,0].reshape(-1,1), T, axis=1)

######################################
# Newton's method
######################################

# Flag used to show or not plots in Newton_reg function (cost and descent direction norm)
show =  False
# Flag used to show trajectories at each iteration of Newton's method
traj_show = False
# Flag used to show or not the descent armijo plot
descent_plot = False

x,u,maxiters = Newton_reg(xref,uref,xinit,uinit,T,dt,maxiters,Qt,Rt,QT,
                          stepsize_0,beta,c,armijo_maxiters,show,traj_show,descent_plot,term_cond=term_cond)

# Optimal trajectory

xstar = x[:,:,maxiters-1]
ustar = u[:,:,maxiters-1]

# Trajectory generation plots

In [ ]:
######################################
# Trajectory generation plots
######################################

ustar[:,-1] = ustar[:,-2] # for plotting purposes
tt_hor = np.linspace(0, tf, T)

# Optimal trajectory and reference curve
traj_gen_plot(tt_hor,xstar,xref,ustar,uref,ns)

# Trajectory tracking with MPC 

In [50]:
#######################################
# Trajectory tracking with mpc (Task4)
#######################################

# weight matrices for the regulator
#Qmpc = np.diag([10000,10000,100,100])
#Rmpc = 0.001*np.eye(ni)
#QTmpc = Qmpc

Qmpc = np.diag([100000,100000,0.1,0.1])
Rmpc = 0.001*np.eye(ni)
QTmpc = 10*Qmpc

# Nominal x0 of the reference trajectory 
x0 = np.array([xstar[0,0],xstar[1,0],xstar[2,0],xstar[3,0]])
# Noise
noise = np.array([0.1, -0.1, 0, 0])
# Perturbed initial condition
x0 = x0 + noise
#Prediction horizon
pred_hor = 5
# Flag to choose the desired mpc function
mpc1 = True

# Real trajectory followed by the robot with mpc
if mpc1:
    xreal_mpc,ureal_mpc = mpc_controller1(xstar,ustar,noise,pred_hor,Qmpc,Rmpc,QTmpc)
else:
    xreal_mpc,ureal_mpc = mpc_controller2(Qmpc,Rmpc,QTmpc,xstar,ustar,x0,T,pred_hor)

# Trajectory tracking with MPC plots

In [ ]:
######################################
# Trajectory tracking with MPC plots
######################################
ureal_mpc[:,-1] = ureal_mpc[:,-2] # for plotting purposes

sim_steps = xreal_mpc.shape[1]
xdes = xstar[:,:sim_steps]
udes = ustar[:,:sim_steps]

# Final time
final_time = sim_steps*dt

time = np.linspace(0, final_time, sim_steps)

# Trajectory tracking plots
traj_track_plot(time,xreal_mpc,xdes,ureal_mpc,udes,ns)

# Tracking error plots
track_error_plot(time,xreal_mpc,xdes,ureal_mpc,udes,ns)